## Introduction

The notebooks provides an example of json event that can be sent to metarank so as to receive the ranked items as a response 

Metarank is a recommendation engine that will be set up by Samagra on a remote desktop. Once it is setup, it gets hosted on a server and recommendations work by sending API ranking event requests to its URLs and the engine returns recommendations for the received events. 

We are trying to get recommendations for content for a user. Hence, for a user we know the userid(phone number) and the list of items to be ranked and the recommendation engine will return a ranked list of items 

A sample ranking event looks like the below: 

In [1]:
{"user": "9583860235", 
               "timestamp": 1641026613708, 
               "items": [{"id": "20220103_paddy_basal_fertilizer_crop", "relevancy": 0},
                         {"id": "20220102_pr_rabi_paddy_line_sowing_transplanting_crop", "relevancy": 0},
                         {"id": "20220102_animal_goat_sheep_bm_livestock", "relevancy": 0}, 
                         {"id": "20220101_animal_goat_sheep_fm_livestock", "relevancy": 0}],
               "id": "2022_01_01-2022_02_28_10596", 
               "event": "ranking", 
               "session": "9583860235", 
               "tenant": "default", "fields": []}

{'user': '9583860235',
 'timestamp': 1641026613708,
 'items': [{'id': '20220103_paddy_basal_fertilizer_crop', 'relevancy': 0},
  {'id': '20220102_pr_rabi_paddy_line_sowing_transplanting_crop',
   'relevancy': 0},
  {'id': '20220102_animal_goat_sheep_bm_livestock', 'relevancy': 0},
  {'id': '20220101_animal_goat_sheep_fm_livestock', 'relevancy': 0}],
 'id': '2022_01_01-2022_02_28_10596',
 'event': 'ranking',
 'session': '9583860235',
 'tenant': 'default',
 'fields': []}

Describing the parts of the json event: 
- event: This is to describe which type of event it is, it should always be ‘ranking’ for ranking events
- user:  The user to which the ranking list was shown. This is the userid (phone number)
- items: List of items that were shown to the user. If we have some previous knowledge of the relevancy of the items, that can be fed here. However, we always consider relevancy as 0. Over here, we should provide all the content names that need to be ranked or recommended
- id:  All ranking events have an associated id which should be unique
- session: This provides the session id for the user. A user can have multiple sessions. In our current model, users and sessions are the same, each user is considered to be having one session only 
- Tenant: This is to  allow for multi-tenancy cases. In our case, it always has the value as ‘default’ 
- Fields:  This is for any additional fields for the ranking event. This is always an empty list in our model for ranking events 


**URL on which metarank gets hosted** :  "http://localhost:8080/rank/xgboost"

Essentially, we  need to figure out how to :

    - Convert the list of content and users that they have for which they need recommendations to the above event format 
    
    - Send the above events as POST request to the URL
    
    - Parse the response from the URL in the required format

## Sending events to the the metarank URL:

### Installing libraries:

In [ ]:
!pip install grequests

In [2]:
import pandas as pd 
import numpy as np 
import os as os
from datetime import datetime
import json
import gzip
import pprint
import grequests
import requests

### Setting metarank URL to send POST request to : 

In [9]:
url_rankings_explain = "http://localhost:8080/rank/xgboost?explain=TRUE"
url_rankings = "http://localhost:8080/rank/xgboost"
url_feedback = "http://localhost:8080/feedback"

#### Defining a sample event: 

In [10]:
sample_ranking = {"user": "9583860237", 
               "timestamp": 1641026613708, 
               "items": [{"id": "20220103_paddy_basal_fertilizer_crop", "relevancy": 0},
                         {"id": "20220102_pr_rabi_paddy_line_sowing_transplanting_crop", "relevancy": 0},
                         {"id": "20220104_pr_pulses_rabi_nutrient_management_crop", "relevancy": 0},
                         {"id": "20220101_pr_rabi_pulses_seed_treatment_pesticide_crop", "relevancy": 0},
                         {"id": "20220101_pr_rabi_paddy_line_sowing_transplanting_crop", "relevancy": 0}, 
                         {"id": "20220103_animal_cattle_buffalo_pp_livestock", "relevancy": 0}, 
                         {"id": "20211204_animal_cattle_buffalo_dm_livestock", "relevancy": 0},
                         {"id": "20220103_animal_goat_sheep_dm_livestock", "relevancy": 0},
                         {"id": "20211204_animal_goat_sheep_fm_livestock", "relevancy": 0}, 
                         {"id": "20220102_animal_chicken_duck_pp_livestock", "relevancy": 0}, 
                         {"id": "20220101_pr_rabi_paddy_seed_treatment_crop", "relevancy": 0}, 
                         {"id": "20220103_pr_paddy_zinc_application_basal_crop", "relevancy": 0}, 
                         {"id": "20220102_animal_cattle_buffalo_dm_livestock", "relevancy": 0}, 
                         {"id": "20220102_pr_rabi_pulses_seed_treatment_biofungicides_biofertilizer_crop", "relevancy": 0},
                         {"id": "20220103_animal_chicken_duck_pp_livestock", "relevancy": 0}, 
                         {"id": "20211203_paddy_seed_treatment_crop", "relevancy": 0}, 
                         {"id": "20220103_pr_rabi_pulses_seed_treatment_biofungicides_biofertilizer_crop", "relevancy": 0},
                         {"id": "20220102_pr_paddy_zinc_application_basal_crop", "relevancy": 0}, 
                         {"id": "20220103_pr_pulses_weed_management_crop", "relevancy": 0}, 
                         {"id": "20220103_pr_rabi_paddy_line_sowing_transplanting_crop", "relevancy": 0}, 
                         {"id": "20220102_pr_pulses_weed_management_crop", "relevancy": 0},
                         {"id": "20220101_pr_rabi_pulses_seeed_treatment_fungicides_crop", "relevancy": 0},
                         {"id": "20220101_animal_cattle_buffalo_fm_livestock", "relevancy": 0}, 
                         {"id": "20220102_animal_goat_sheep_bm_livestock", "relevancy": 0}, 
                         {"id": "20220101_animal_goat_sheep_fm_livestock", "relevancy": 0}],
               "id": "2022_01_01-2022_02_28_10596", 
               "event": "ranking", 
               "session": "9583860237", 
               "tenant": "default", "fields": []}


#### Sending a POST request with the sample event: 

The response from the URL is stored in the variable response

In [11]:
payload = json.dumps(sample_ranking)
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url_rankings, headers=headers, data=payload,timeout=10)

#### Received response :

Below, we can see that we have received a ranked list of the items that we sent to metarank along with their score for the user

In [12]:
response

<Response [504]>

In [8]:
json1_data = json.loads(response.text)
json1_data['items']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Sending events for multiple users at once :

We below define a list of json events such that each event is for a different user and contains single/multiple items to be ranked 

In [8]:
send_req = [{'user': '6000396560',
  'items': [{'id': '20220204_paddy_ysb_dpm_crop', 'relevancy': 0}],
  'id': '2022_03_010',
  'event': 'ranking',
  'session': '6000396560',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6000962788',
  'items': [{'id': '20220204_mustard_aphid_dpm_crop', 'relevancy': 0},
   {'id': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
    'relevancy': 0},
   {'id': '20220301_animal_cattle_buffalo_dm_livestock', 'relevancy': 0}],
  'id': '2022_03_011',
  'event': 'ranking',
  'session': '6000962788',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6001385413',
  'items': [{'id': '20220203_animal_goat_sheep_dm_livestock', 'relevancy': 0},
   {'id': '20220301_animal_goat_sheep_pp_livestock', 'relevancy': 0},
   {'id': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
    'relevancy': 0}],
  'id': '2022_03_012',
  'event': 'ranking',
  'session': '6001385413',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6001401077',
  'items': [{'id': '20220301_paddy_fertilizer_pi_crop', 'relevancy': 0}],
  'id': '2022_03_013',
  'event': 'ranking',
  'session': '6001401077',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6001755806',
  'items': [{'id': '20220204_okra_fertilizer_crop', 'relevancy': 0},
   {'id': '20220303_okra_ymv_dpm_crop', 'relevancy': 0},
   {'id': '20220204_brinjal_epilachna_beetle_dpm_crop', 'relevancy': 0}],
  'id': '2022_03_014',
  'event': 'ranking',
  'session': '6001755806',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6002555784',
  'items': [{'id': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
    'relevancy': 0}],
  'id': '2022_03_015',
  'event': 'ranking',
  'session': '6002555784',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6002625472',
  'items': [{'id': '20220203_animal_goat_sheep_dm_livestock', 'relevancy': 0},
   {'id': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
    'relevancy': 0}],
  'id': '2022_03_016',
  'event': 'ranking',
  'session': '6002625472',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6003094533',
  'items': [{'id': '20220204_paddy_ysb_dpm_crop', 'relevancy': 0},
   {'id': '20220302_paddy_ysb_dpm_crop', 'relevancy': 0},
   {'id': '20220203_animal_cattle_buffalo_hm_livestock', 'relevancy': 0},
   {'id': '20220303_paddy_pi_fertilizer_crop', 'relevancy': 0}],
  'id': '2022_03_017',
  'event': 'ranking',
  'session': '6003094533',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6005070448',
  'items': [{'id': '20220301_pulses_aphid_dpm_crop', 'relevancy': 0},
   {'id': '20220204_pulses_oilseed_tobacco_caterpillar_dpm_crop',
    'relevancy': 0},
   {'id': '20220302_blackgram_greengram_aphid_dpm_crop', 'relevancy': 0}],
  'id': '2022_03_018',
  'event': 'ranking',
  'session': '6005070448',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000},
 {'user': '6005117243',
  'items': [{'id': '20220301_animal_cattle_buffalo_dm_livestock',
    'relevancy': 0}],
  'id': '2022_03_019',
  'event': 'ranking',
  'session': '6005117243',
  'tenant': 'default',
  'fields': [],
  'timestamp': 1646179190000}]

#### Sending the above json event as POST request to Metarank:

In [10]:

headers = {
  'Content-Type': 'application/json'
}

response = [grequests.post(url_rankings, headers=headers, data=json.dumps(a_url),timeout=10) for a_url in send_req]
responses1 = grequests.map(response)

In [11]:
all_ranks =  [json.loads(x.text)['items'] for x in responses1 ]

#### Results for one user: 

In [12]:
all_ranks[1]

[{'item': '20220204_mustard_aphid_dpm_crop',
  'score': -0.27890539169311523,
  'features': []},
 {'item': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
  'score': -0.27890539169311523,
  'features': []},
 {'item': '20220301_animal_cattle_buffalo_dm_livestock',
  'score': -0.27890539169311523,
  'features': []}]

####  Results for all users:

In [55]:
all_ranks

[[{'item': '20220204_paddy_ysb_dpm_crop',
   'score': -0.28562623262405396,
   'features': []}],
 [{'item': '20220204_mustard_aphid_dpm_crop',
   'score': -0.28562623262405396,
   'features': []},
  {'item': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
   'score': -0.28562623262405396,
   'features': []},
  {'item': '20220301_animal_cattle_buffalo_dm_livestock',
   'score': -0.3230266571044922,
   'features': []}],
 [{'item': '20220203_animal_goat_sheep_dm_livestock',
   'score': -0.28562623262405396,
   'features': []},
  {'item': '20220301_animal_goat_sheep_pp_livestock',
   'score': -0.28562623262405396,
   'features': []},
  {'item': '20220303_animal_cattle_buffalo_goat_sheep_pp_livestock',
   'score': -0.28562623262405396,
   'features': []}],
 [{'item': '20220301_paddy_fertilizer_pi_crop',
   'score': -0.28562623262405396,
   'features': []}],
 [{'item': '20220204_okra_fertilizer_crop',
   'score': -0.28562623262405396,
   'features': []},
  {'item': '20220303_okra_y